In [1]:
!pip install -U tensorflow_decision_forests

     |████████████████████████████████| 13.4 MB 930 kB/s            
     |████████████████████████████████| 497.5 MB 21 kB/s              
     |████████████████████████████████| 462 kB 39.1 MB/s            
     |████████████████████████████████| 2.1 MB 39.2 MB/s            
     |████████████████████████████████| 1.4 MB 42.0 MB/s            
     |████████████████████████████████| 5.8 MB 57.7 MB/s            
     |████████████████████████████████| 14.5 MB 48.8 MB/s            
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.2
    Uninstalling tensorflow-2.6.2:
      Successfully uninstalled tensorflow-2.6.2
ERROR: pip's dependency re

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import urllib
import math

from tensorflow import keras
from tensorflow.keras.layers import *

import tensorflow_decision_forests as tfdf


from wurlitzer import sys_pipes

In [3]:
df = pd.read_csv("../input/flight-fare/Data_Train.csv")
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
df['journey_day'] = pd.to_datetime(df.Date_of_Journey,format='%d/%m/%Y').dt.day
df['journey_month'] = pd.to_datetime(df.Date_of_Journey,format='%d/%m/%Y').dt.month
df.drop(['Date_of_Journey'],axis=1,inplace=True)

In [5]:
duration = list(df["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))  

In [6]:
df['Duration_hrs'] = duration_hours
df['Duration_mins'] = duration_mins
df.drop(['Duration'],axis=1,inplace=True)

In [7]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,journey_day,journey_month,Duration_hrs,Duration_mins
0,IndiGo,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,non-stop,No info,3897,24,3,2,50
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,2 stops,No info,7662,1,5,7,25
2,Jet Airways,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,2 stops,No info,13882,9,6,19,0
3,IndiGo,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,1 stop,No info,6218,12,5,5,25
4,IndiGo,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,1 stop,No info,13302,1,3,4,45


In [8]:
df['dep_hr'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['dep_min'] = pd.to_datetime(df['Dep_Time']).dt.hour
df.drop(['Dep_Time'],axis=1,inplace=True)

df['Arr_hr'] = pd.to_datetime(df['Arrival_Time']).dt.hour
df['Arr_min'] = pd.to_datetime(df['Arrival_Time']).dt.hour
df.drop(['Arrival_Time'],axis=1,inplace=True)

In [9]:
df.drop(['Route','Additional_Info'],axis=1,inplace=True)

In [10]:
df.replace({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4},inplace=True)

In [11]:
df.head(2)

,Airline,Source,Destination,Total_Stops,Price,journey_day,journey_month,Duration_hrs,Duration_mins,dep_hr,dep_min,Arr_hr,Arr_min
0,IndiGo,Banglore,New Delhi,0.0,3897,24,3,2,50,22,22,1,1
1,Air India,Kolkata,Banglore,2.0,7662,1,5,7,25,5,5,13,13


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Airline        10683 non-null  object 
 1   Source         10683 non-null  object 
 2   Destination    10683 non-null  object 
 3   Total_Stops    10682 non-null  float64
 4   Price          10683 non-null  int64  
 5   journey_day    10683 non-null  int64  
 6   journey_month  10683 non-null  int64  
 7   Duration_hrs   10683 non-null  int64  
 8   Duration_mins  10683 non-null  int64  
 9   dep_hr         10683 non-null  int64  
 10  dep_min        10683 non-null  int64  
 11  Arr_hr         10683 non-null  int64  
 12  Arr_min        10683 non-null  int64  
dtypes: float64(1), int64(9), object(3)
memory usage: 1.1+ MB


In [13]:
label = "Price"

In [14]:
def split_dataset(dataset, test_ratio=0.30):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

7415 examples in training, 3268 examples for testing.


In [15]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label, tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label, tfdf.keras.Task.REGRESSION)

/opt/conda/lib/python3.7/site-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
2022-03-12 11:11:57.443575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-03-12 11:11:57.469975: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the 

In [16]:
model_rf = tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.REGRESSION,num_trees=500,max_depth=8,
                                               maximum_training_duration_seconds=100.0,verbose=1)
model_rf.compile(metrics=["mse", "mape"])
with sys_pipes():
    model_rf.fit(x=train_ds)

Use /tmp/tmph3nsgv4e as temporary training directory
Starting reading the dataset
8/8 [==============================] - ETA: 0s 
Dataset read in 0:00:04.601975
Training model
Model trained in 0:00:03.297996
Compiling model


[INFO kernel.cc:1153] Loading model from path
[INFO abstract_model.cc:1063] Engine "GradientBoostedTreesOptPred" built
[INFO kernel.cc:1001] Use fast generic engine


8/8 [==============================] - 8s 487ms/step
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [17]:
evaluation = model_rf.evaluate(test_ds, return_dict=True)

print(evaluation)
print(f"MSE: {evaluation['mse']:.2f}")
print(f"RMSE: {math.sqrt(evaluation['mse']):.2f}")
print(f"MAPE: {evaluation['mape']:.2f}")

4/4 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - mse: 4263760.0000 - mape: 13.7928
{'loss': 0.0, 'mse': 4263760.0, 'mape': 13.792808532714844}
MSE: 4263760.00
RMSE: 2064.89
MAPE: 13.79


In [18]:
predictions = model_rf.predict(test_ds)
y_true      = test_ds_pd["Price"]

from sklearn.metrics import r2_score
r2 = r2_score(y_true, predictions)
print("The r2_square score is %.5f" % r2 )

The r2_square score is 0.81003


In [19]:
 with open("plot.html", "w") as f:
    f.write(tfdf.model_plotter.plot_model(model_rf))

In [20]:
inspector = model_rf.make_inspector()
print(inspector.num_trees())

103


In [21]:
inspector.features()

["Airline" (4; #0),
 "Arr_hr" (1; #1),
 "Arr_min" (1; #2),
 "Destination" (4; #3),
 "Duration_hrs" (1; #4),
 "Duration_mins" (1; #5),
 "Source" (4; #6),
 "Total_Stops" (1; #7),
 "dep_hr" (1; #8),
 "dep_min" (1; #9),
 "journey_day" (1; #10),
 "journey_month" (1; #11)]

In [22]:
inspector.variable_importances()

{'SUM_SCORE': [("Duration_hrs" (1; #4), 317593418625.1678),
  ("Airline" (4; #0), 120601786974.48021),
  ("journey_month" (1; #11), 76463772113.70587),
  ("journey_day" (1; #10), 70396109025.6493),
  ("Total_Stops" (1; #7), 19037286451.994385),
  ("Destination" (4; #3), 18072398084.64113),
  ("Duration_mins" (1; #5), 13734466117.29792),
  ("Source" (4; #6), 10774617008.37079),
  ("Arr_min" (1; #2), 9796338003.015808),
  ("Arr_hr" (1; #1), 9146087857.662254),
  ("dep_min" (1; #9), 6849243669.327583),
  ("dep_hr" (1; #8), 6614811544.105774)],
 'NUM_NODES': [("journey_day" (1; #10), 1231.0),
  ("Airline" (4; #0), 972.0),
  ("Duration_mins" (1; #5), 900.0),
  ("journey_month" (1; #11), 836.0),
  ("Duration_hrs" (1; #4), 812.0),
  ("Arr_hr" (1; #1), 468.0),
  ("Arr_min" (1; #2), 427.0),
  ("dep_min" (1; #9), 418.0),
  ("dep_hr" (1; #8), 407.0),
  ("Destination" (4; #3), 387.0),
  ("Total_Stops" (1; #7), 272.0),
  ("Source" (4; #6), 261.0)],
 'NUM_AS_ROOT': [("Duration_hrs" (1; #4), 37.0),
 